<a href="https://colab.research.google.com/github/sallywang147/SCInvarinfer/blob/main/%5B3_9%5D_T5_for_smart_contracts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentencepiece
!pip install transformers
!pip install rich[jupyter]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 24.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.7/238.7 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 67.7 MB/s eta 0:00:00
  Attempting uninstall: pygments
    Found existing installation

In [2]:
from google.colab import auth
from google.auth import default
from numpy import random
import pandas as pd
import gspread
import gc
#autenticating to google
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
#defining my worksheet
worksheet = gc.open('invariants_line_number').sheet1
#get_all_values gives a list of rows
rows = worksheet.get_all_values()
#Convert to a DataFrame 
cols = ['Source', 'Target', 'Verify_Success']
df = pd.DataFrame(rows, columns=cols)

In [3]:
df

,Source,Target,Verify_Success
0,Source,Target,Verify Success
1,1 pragma solidity >=0.4.24 <0.6.0;\n\n2 contra...,14+ assert (y == x + 4);,
2,1 // SPDX-License-Identifier: MIT\n2 pragma ex...,"109+ require(admin == msg.sender, ""Ownable: c...",
3,1 pragma solidity >=0.4.24 <0.6.0;\n\n2 contra...,7+ assert (y == x + 2);,
4,1 pragma solidity >=0.4.24 <0.6.0;\n2 contract...,20+ assert(funcA2(funcA1())==12);\n20+ assert(...,
...,...,...,...
83,1 pragma solidity >=0.4.24 <0.6.0;\n2 contract...,21+ assert(funcA2(funcA1())==12);\n22+ asser...,
84,"1 pragma solidity ^0.5.0;\n\n2 import ""./IERC2...",11+ require(msg.sender != address(0));\n22+ _...,
85,\n1 pragma solidity ^0.4.11;\n\n2 interface to...,12 require(msg.sender == owner);\n63 ...,
86,\n1 pragma solidity ^0.4.2;\n2 contract owned ...,7+ assert(msg.sender == owner);\n125+ assert(...,


In [4]:
# Importing libraries
import os
import numpy as np
import pandas as pd
import plotly.express as px
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler


# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

from rich.table import Column, Table
from rich import box
from rich.console import Console

# define a rich console logger
console=Console(record=True)

def display_df(df):
  """display dataframe in ASCII format"""

  console=Console()
  table = Table(Column("source_text", justify="center" ), Column("target_text", justify="center"), title="Sample Data",pad_edge=False, box=box.ASCII)

  for i, row in enumerate(df.values.tolist()):
    table.add_row(row[0], row[1])

  console.print(table)

def plot_loss(index_list, loss_list):
  results = {
      "epochs": index_list,
      "cross entropy loss": loss_list,
  }
  df = pd.DataFrame(results)
  fig = px.line(df, x ="epochs", y="cross entropy loss",  title="Evaluation")
  fig.show()

training_logger = Table(Column("Epoch", justify="center" ), 
                        Column("Steps", justify="center"),
                        Column("Cross Entropy Loss", justify="center"), 
                        title="Training Status",pad_edge=False, box=box.ASCII)


In [5]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [6]:
class YourDataSetClass(Dataset):
  """
  Creating a custom dataset for reading the dataset and 
  loading it into the dataloader to pass it to the neural network for finetuning the model

  """

  def __init__(self, dataframe, tokenizer, source_len, target_len, source_text, target_text):
    self.tokenizer = tokenizer
    self.data = dataframe
    self.source_len = source_len
    self.summ_len = target_len
    self.target_text = self.data[target_text]
    self.source_text = self.data[source_text]

  def __len__(self):
    return len(self.target_text)

  def __getitem__(self, index):
    source_text = str(self.source_text[index])
    target_text = str(self.target_text[index])

    #cleaning data so as to ensure data is in string type
    source_text = ' '.join(source_text.split())
    target_text = ' '.join(target_text.split())
    
    source = self.tokenizer.batch_encode_plus([source_text], \
                                              max_length= self.source_len, \
                                              pad_to_max_length=True, \
                                              truncation=True, \
                                            #  padding='longest',\
                                              return_tensors='pt')
    target = self.tokenizer.batch_encode_plus([target_text], \
                                              max_length= self.summ_len, \
                                              pad_to_max_length=True, \
                                              truncation=True, \
                                           #   padding='longest', \
                                              return_tensors='pt')   
    source_ids = source['input_ids'].squeeze()
    source_mask = source['attention_mask'].squeeze()
    target_ids = target['input_ids'].squeeze()
    target_mask = target['attention_mask'].squeeze()

    return {
        'source_ids': source_ids.to(dtype=torch.long), 
        'source_mask': source_mask.to(dtype=torch.long), 
        'target_ids': target_ids.to(dtype=torch.long),
        'target_ids_y': target_ids.to(dtype=torch.long)
    }

In [7]:
def train(epoch, tokenizer, model, device, loader, optimizer):

  """
  Function to be called for training with the parameters passed from main function

  """

  model.train()
  loss_list = []
  for _, data in enumerate(loader, 0):
    y = data['target_ids'].to(device, dtype = torch.long)
    y_ids = y[:, :-1].contiguous()
    lm_labels = y[:, 1:].clone().detach()
    lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
    ids = data['source_ids'].to(device, dtype = torch.long)
    mask = data['source_mask'].to(device, dtype = torch.long)
    outputs = model(input_ids = ids, attention_mask = mask, \
                    decoder_input_ids=y_ids, labels=lm_labels)  
    total_loss = float(outputs[0].item())
    loss_list.append(total_loss)
    if _%10==0:
      training_logger.add_row(str(epoch), str(_), str(total_loss))
      console.print(training_logger)

    optimizer.zero_grad()
    outputs[0].backward()
    optimizer.step()
    return loss_list



In [8]:
def validate(epoch, tokenizer, model, device, loader):

  """
  Function to evaluate model for predictions

  """
  model.eval()
  predictions = []
  actuals = []
  with torch.no_grad():
      for _, data in enumerate(loader, 0):
          y = data['target_ids'].to(device, dtype = torch.long)
          ids = data['source_ids'].to(device, dtype = torch.long)
          mask = data['source_mask'].to(device, dtype = torch.long)

          generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask, 
              max_length=700, 
              num_beams=2,
              repetition_penalty=2.5, 
              length_penalty=5.0, 
              early_stopping=True
              )
          print("generated token length: \n", len(generated_ids[0]))
          preds = [tokenizer.decode(g, skip_special_tokens=True, \
                                    clean_up_tokenization_spaces=True) for g in generated_ids]
          target = [tokenizer.decode(t, skip_special_tokens=True, \
                                     clean_up_tokenization_spaces=True)for t in y]
          if _%10==0:
              console.print(f'Completed {_}')
          predictions.extend(preds)
          actuals.extend(target)
  print('predictions: \n', predictions)
  print('actuals: \n', actuals)
  return predictions, actuals

In [9]:
import gc
def T5Trainer(dataframe, source_text, target_text, model_params, output_dir="./output/"):
  
  """
  T5 trainer

  """

  # Set random seeds and deterministic pytorch for reproducibility
  torch.manual_seed(model_params["SEED"]) # pytorch random seed
  np.random.seed(model_params["SEED"]) # numpy random seed
  torch.backends.cudnn.deterministic = True

  # logging
  console.log(f"""[Model]: Loading {model_params["MODEL"]}...\n""")

  # tokenzier for encoding the text
  tokenizer = T5Tokenizer.from_pretrained(model_params["MODEL"])

  gc.collect()
  torch.cuda.empty_cache()
  os.environ['CUDA_VISIBLE_DEVICES']='0, 1, 2, 3'
  os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
  # Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary. 
  # Further this model is sent to device (GPU/TPU) for using the hardware.
  model = T5ForConditionalGeneration.from_pretrained(model_params["MODEL"])
  model = model.to(device)
  
  # logging
  console.log(f"[Data]: Reading data...\n")

  # Importing the raw dataset
  dataframe = dataframe[[source_text,target_text]]
  display_df(dataframe.head(2))

  
  # Creation of Dataset and Dataloader
  # Defining the train size. So 80% of the data will be used for training and the rest for validation. 
  train_size = 0.8
  train_dataset=dataframe.sample(frac=train_size,random_state = model_params["SEED"])
  val_dataset=dataframe.drop(train_dataset.index).reset_index(drop=True)
  train_dataset = train_dataset.reset_index(drop=True)

  console.print(f"FULL Dataset: {dataframe.shape}")
  console.print(f"TRAIN Dataset: {train_dataset.shape}")
  console.print(f"TEST Dataset: {val_dataset.shape}\n")


  # Creating the Training and Validation dataset for further creation of Dataloader
  training_set = YourDataSetClass(train_dataset, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text, target_text)
  val_set = YourDataSetClass(val_dataset, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text, target_text)


  # Defining the parameters for creation of dataloaders
  train_params = {
      'batch_size': model_params["TRAIN_BATCH_SIZE"],
      'shuffle': True,
      'num_workers': 0
      }


  val_params = {
      'batch_size': model_params["VALID_BATCH_SIZE"],
      'shuffle': False,
      'num_workers': 0
      }


  # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
  training_loader = DataLoader(training_set, **train_params)
  val_loader = DataLoader(val_set, **val_params)


  # Defining the optimizer that will be used to tune the weights of the network in the training session. 
  optimizer = torch.optim.Adam(params = model.parameters(), lr=model_params["LEARNING_RATE"])


  # Training loop
  console.log(f'[Initiating Fine Tuning]...\n')

  loss_result = []
  epoch_list = []
  for epoch in range(model_params["TRAIN_EPOCHS"]):
      loss = train(epoch, tokenizer, model, device, training_loader, optimizer)
      loss_result.extend(loss)
      epoch_list.append(epoch)

  plot_loss(epoch_list, loss_result)
  console.log(f"[Saving Model]...\n")
  #Saving the model after training
  path = os.path.join(output_dir, "model_files")
  model.save_pretrained(path)
  tokenizer.save_pretrained(path)


  # evaluating test dataset
  console.log(f"[Initiating Validation]...\n")
  for epoch in range(model_params["VAL_EPOCHS"]):
    predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
    final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
    final_df.to_csv(os.path.join(output_dir,'predictions.csv'))
  
  console.save_text(os.path.join(output_dir,'logs.txt'))
  
  console.log(f"[Validation Completed.]\n")
  console.print(f"""[Model] Model saved @ {os.path.join(output_dir, "model_files")}\n""")
  console.print(f"""[Validation] Generation on Validation data saved @ {os.path.join(output_dir,'predictions.csv')}\n""")
  console.print(f"""[Logs] Logs saved @ {os.path.join(output_dir,'logs.txt')}\n""")

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
model_params={
    "MODEL":"t5-small",             # model_type: t5-large
    "TRAIN_BATCH_SIZE": 8,          # training batch size: 
    "VALID_BATCH_SIZE":8,          # validation batch size
    "TRAIN_EPOCHS":226,              # number of training epochs:20 seems optimal based on experiments
    "VAL_EPOCHS":1,                # number of validation epochs
    "LEARNING_RATE":0.001,          # learning rate: can be 1e-4
    "MAX_SOURCE_TEXT_LENGTH": 1000,  # max length of source text
    "MAX_TARGET_TEXT_LENGTH": 1000,   # max length of target text
    "SEED": random.randint(1000)    # randomized seeds to shuffle test set

}

In [12]:
#To solve CUDA out of memory error
import gc
gc.collect()
torch.cuda.empty_cache()
os.environ['CUDA_VISIBLE_DEVICES']='0, 1, 2, 3'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [13]:
T5Trainer(dataframe=df, source_text="Source", target_text="Target", model_params=model_params, output_dir="outputs")

[18:46:10] [Model]: Loading t5-small...                                           <ipython-input-9-c6bf7c73f73a>:15
                                                                                                                   

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


[18:46:21] [Data]: Reading data...                                                <ipython-input-9-c6bf7c73f73a>:30
                                                                                                                   

                                              Sample Data                                              
+-----------------------------------------------------------------------------------------------------+
|                               source_text                                 |       target_text       |
|---------------------------------------------------------------------------+-------------------------|
|                                  Source                                   |          Target         |
|                    1 pragma solidity >=0.4.24 <0.6.0;                     | 14+ assert (y == x + 4);|
|                                                                           |                         |
|                             2 contract Bar {                              |                         |
|                                                                           |                         |
|                      3     constructor () public {}                       |                         |
|                                                                           |                         |
|       4     function baz(uint p) public pure returns (uint ret) {         |                         |
|                          5         ret = p + 2;                           |                         |
|                                 6     }                                   |                         |
|                                                                           |                         |
|                                   7 }                                     |                         |
|                    8 contract ExternalFunctionCall {                      |                         |
|                                                                           |                         |
|    9     function foo(uint x, Bar y) public pure returns (uint ret) {     |                         |
|                      10         ret = x + y.baz(2);                       |                         |
|                                 11     }                                  |                         |
|                                                                           |                         |
|12     function testExternalFunctionCall(uint x) public returns (uint y) { |                         |
|                     13         Bar bar = new Bar();                       |                         |
|                     14         y = this.foo(x, bar);                      |                         |
|                                 15     }                                  |                         |
|                                                                           |                         |
|                                   16 }                                    |                         |
+-----------------------------------------------------------------------------------------------------+

FULL Dataset: (88, 2)

...

generated token length: 
 96
generated token length: 
 44
predictions: 
 ['>= x); 3+ assert (y >= x);', '7+ assert(x == 0); 22+ assert (x == 0); 22+ assert (y == 0); 22+ assert (y == 0); 22+ assert (y == 0); 22+ assert (y == 0); 22+ assert (y == 0);', '- amount); 6+ assert(_totalSupply - amount); 8+ assert(msg.sender, amount);', ': _msgScan.com on 2021-08-30 3 * assert(a == 0 || _msgScan.com on 2021-07-30 3 * assert(b = balanceOf(address(b = address(0));', 'isContract(from), "ERC20/ERC20.sol"); 92+ require(_addr!= 0);', ', "EthCrossChainData is not the owner"); 92+ require(_eccd.getCurEpochConPubKeyList, "EthCrossChainData is not the owner");', ', "provided tokenAddress is not whitelisted"); 125+ require(_contractWhitelist[address(tokenAddress)], "provided tokenAddress is not whitelisted");', '; 19+ assert (a == 1); 20+ assert (b == 55); 21+ assert (a == 5);', 'B  1 pragma solidity >= a + 2;', 'B  10+ assert (x == a + 2);', '>= tokenOut, "ERC20:INSUFF_OUTPUT"); 92+ require(canTransfer,

[18:48:23] [Validation Completed.]                                               <ipython-input-9-c6bf7c73f73a>:105
                                                                                                                   

[Model] Model saved @ outputs/model_files

[Validation] Generation on Validation data saved @ outputs/predictions.csv

[Logs] Logs saved @ outputs/logs.txt

In [14]:

test_params = {
      'batch_size': 16,
      'shuffle': False,
      'num_workers': 0
      }

class TestDataSetClass(Dataset):
  """
  Creating a custom dataset for reading the dataset and 
  loading it into the dataloader to pass it to the neural network for finetuning the model

  """

  def __init__(self, dataframe, tokenizer, source_len, source_text):
    self.tokenizer = tokenizer
    self.data = dataframe
    self.source_len = source_len
    self.source_text = self.data[source_text]

  def __len__(self):
    return len(self.source_text)

  def __getitem__(self, index):
    source_text = str(self.source_text[index])

    #cleaning data so as to ensure data is in string type
    source_text = ' '.join(source_text.split())
    print('source_text: ', source_text)
    source = self.tokenizer.batch_encode_plus([source_text], \
                                              max_length=self.source_len, \
                                              pad_to_max_length=True, \
                                              truncation=True, \
                                              padding="max_length",\
                                              return_tensors='pt')

    source_ids = source['input_ids'].squeeze()
    source_mask = source['attention_mask'].squeeze()

    return {
        'source_ids': source_ids.to(dtype=torch.long), 
        'source_mask': source_mask.to(dtype=torch.long), 
    }

In [215]:
test_contract = 'reentrancy.sol'

In [216]:

import gc
#let's get the trained model and never-seen test contracts 
def initialize():
  gc = gspread.authorize(creds)
  trained_model = T5ForConditionalGeneration.from_pretrained("outputs/model_files")
  tokenizer = T5Tokenizer.from_pretrained("t5-small")
  col = ['Test']
  f = open(f'/content/drive/MyDrive/experiments/baseline benchmark/{test_contract}', "r")
  file = f.read()
  test_df = pd.DataFrame([file], columns=col)
  #Convert to a DataFrame 
  return test_df, trained_model, tokenizer

#test how the model performs on never-seen test contracts
def generate(df, model, tokenizer):
   model.eval()
   test_val = TestDataSetClass(df, tokenizer, source_len=512, source_text="Test")
   test_loader = DataLoader(test_val, **test_params)
   predictions = []
   with torch.no_grad():
      for _, data in enumerate(test_loader, 0):
          ids = data['source_ids']
          mask = data['source_mask'] #.to(device, dtype = torch.long)

          generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask, 
              max_length=150, 
              num_beams=2,
              repetition_penalty=2.5, 
              length_penalty=1.0, 
              early_stopping=True
              )
          preds = [tokenizer.decode(g, skip_special_tokens=True, \
                                    clean_up_tokenization_spaces=True) for g in generated_ids]
          print("preds: ", preds)
          if _%10==0:
              console.print(f'Completed {_}')
          predictions.extend(preds)
         # print('predictions: \n', predictions)
   return predictions



In [217]:
df2, trained_model, tokenizer = initialize()

In [218]:
out = generate(df2, trained_model, tokenizer)
print(out)

source_text:  1 pragma solidity ^0.6.1; 2 contract Reentrency { 3 uint frames = 0; 4 A a; 5 function init(uint x) public { 6 if (x*2 - 5 == 39) // x == 22 7 if (a == A(address(0x0))) 8 a = new A(); 9 } 10 function enter(uint x, uint y) public { 11 if (frames > 1) 12 return; 13 frames += 1; 14 if (x*3 + 22 == 25) // x == 1 15 if (a != A(address(0x0))) 16 a.enter(address(this), x, y); 17 frames -= 1; 18 } 19 function check() public returns (int) { 20 if (frames == 2) 21 return 0; // test::coverage 22 else if (frames == 1) 23 return -1; // test::coverage 24 else 25 return -2; // test::coverage 26 } 27 } 28 // Some dummy contract 29 contract A { 30 function enter(address c, uint x, uint y) public { 31 uint key = 1000000; 32 if (y+10 == key*10) 33 Reentrency(c).check(); 34 else 35 Reentrency(c).enter(x, y + key); 36 } 37 }
preds:  ['(a == x, y)); 32+ assert(a == 0 || a == 0);']


Completed 0

['(a == x, y)); 32+ assert(a == 0 || a == 0);']


In [219]:
def get_line_and_invariant(prediction):  
  line_df = []
  for item in prediction:   
    for line in item.split(';'):
      line_df.append(line)
  invariants = []
  for i in range(len(line_df)-1): 
      letter_index = 0
      letter = line_df[i][0]
      line_number = ''
      invariant = ''
      while letter != '+':  
        line_number += letter 
        letter_index += 1  
        if letter_index > (len(line_df[i]) - 1):
          break        
        letter = line_df[i][letter_index]        
        if letter_index >= 5:
          line_number = ''        
          break 
      if letter_index >= 5:
        line_number = '' 
        invariant = line_df[i][0:len(line_df[i])] 
      else: 
        invariant = line_df[i][letter_index+1:len(line_df[i])]
      invariants.append([line_number.replace(" ", ""), invariant])
  return invariants

def insert_and_annotate(test_contract, invariants): 
  new_template = open(f'/content/drive/MyDrive/experiments/evaluation/T5_annotated_contracts/{test_contract}(annotated)', "w")
  for line in df2['Test'][0].split('\n'): 
      if line != '':   
        for item in invariants: 
          if item[0] == line[:3].replace(" ", "") and item[0] != '':            
            comment = '//[SCInvarinfer] inserting inferred invarint below: \n'
            line = '\n'+ comment + line + '\n' + '\t' + item[1] + ';'
            new_template.write(line + '\n')    
      new_template.write(line + '\n')
      print(line)
  new_template.close()

In [220]:
invariants = get_line_and_invariant(out)

def save_invariants(test_contract, invariants):
  new_template = open(f'/content/drive/MyDrive/experiments/evaluation/T5_inferred_invariants/{test_contract}(invariants)', "w")
  for line in invariants:       
      new_template.write(str(line) + '\n')
      print(line)
  new_template.close()


In [221]:
save_invariants(test_contract, invariants)

['', '(a == x, y))']
['32', ' assert(a == 0 || a == 0)']


In [222]:
insert_and_annotate(test_contract, invariants)

1 pragma solidity ^0.6.1;


2 contract Reentrency {
3     uint frames = 0;
4     A a;

5     function init(uint x) public {
6         if (x*2  - 5 == 39) // x == 22
7             if (a == A(address(0x0)))
8                 a = new A();
9     }

10     function enter(uint x, uint y) public {
11         if (frames > 1)
12             return;

13         frames += 1;
14         if (x*3 + 22 == 25) // x == 1
15             if (a != A(address(0x0)))
16                 a.enter(address(this), x, y);
17         frames -= 1;
18     }

19     function check() public returns (int) {
20         if (frames == 2)
21             return 0; // test::coverage
22         else if (frames == 1)
23             return -1; // test::coverage
24         else
25             return -2; // test::coverage
26     }

27 }

28 // Some dummy contract
29 contract A {

30     function enter(address c, uint x, uint y) public {
31         uint key = 1000000;

//[SCInvarinfer] inserting inferred invarint below: 
32         

In [22]:
#for downloading purpose 
#!zip -r /content/T5-model.zip /content/outputs/model_files

In [23]:
#from google.colab import files
#files.download("/content/T5-model.zip")